# Conditionals and Loops in Ansible
---

### Conditionals
Conditionals in Ansible work almost identically to how you'd expect them to. That is, when appended to a task, if the condition or conditions are met the task will execute. Conditionals can also be applied to blocks of tasks and included tasks, as can be seen under **Further Explanation**.
- Achieved using `when`
- When used in conjunction with loops, the condition will be checked on each iteration

### Loops
Loops in Ansible are unique in that there is only one kind of loop that is available to the user. Loops in Ansible work in the following way:
1. You must append the loop to a task in Ansible
1. You must give the loop a list of items over which you wish to iterate
1. For each item in this list, the task will execute


- You can use the `loop` or `with_items` keywords
- Can be used in conjunction with conditionals
- _Cannot_ be used on `block`
- Can be used on `include_tasks`

## Demos
---

### Conditionals

Conditionals in Ansible, indicated by the `when` under each task, are actually quite simple as can be seen below.

> Run the playbook below
> - Variables showing the OS TYpe and Distriubution printed
> - Only the taks that matches your OS will be printed


In [ ]:
wpb "ansible_conditionals.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    debug_script: True

  tasks:
    - name: Debug Message Controlled by Conditional
      debug:
        msg:
          - "The time of your run was {{ ansible_date_time.iso8601 }} "
          - "The OS Type is {{ ansible_system }}"
          - "The OS Distribution is {{ ansible_distribution }}"
          - "The OS Major Version is {{ ansible_distribution_major_version }}"
      when: debug_script

    - name: Execute a set of tasks when using Ubuntu 22
      debug:
        msg: "Run a linux tasks for Ubuntu"
      when:
        - 'ansible_distribution == "Ubuntu"'
        - 'ansible_distribution_major_version == "22"'

    - name: Execute a set of tasks when Ubuntu 24
      debug:
        msg: "Run a linux tasks for Ubuntu 24"
      when:
        - 'ansible_distribution == "Ubuntu"'
        - 'ansible_distribution_major_version == "24"'
...
############################################################

ansible-playbook "ansible_conditionals.yml"

### Loops and Combining the Two

Loops in Ansible can be executed one of two ways: using `loop` or `with_items`. You are also able to combine loops and conditionals in Ansible. Take a look at the playbook below and note the different ways in which loops are run.

**1. Run the playbook below as is**

> Note:
> - loops in Ansible require that a list be provided
> - the list can be provided in the tasks or from a variable list
> - the variable list can be preset or the output of another command

**2. Run the playbook below with the conditionals uncommented**

> Note:
> - Uncomment     # when: item == "{{ my_user_id }}"
> - the conditional will be checked for validity on each iteration of the loop
> - the second run only 1 account is created


In [ ]:
wpb "ansible_conds_loops1.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    user_id_list:
        - "phcr"
        - "mcpu"
        - "pdeg"
        - "nfgx"
    my_shell: "/bin/false"  # just using /bin/false as I dont want those accounts to be logged in
    debug_playbook: True
    my_user_id: "phcr"

  tasks:

  - name: Make sure all the users accounts deleted
    user:
      name: "{{ item }}"
      state: absent
      remove: true
    with_items:
        - "phcr"
        - "mcpu"
        - "pdeg"
        - "nfgx"
    become: true

  - name: a debug block of tasks
    block:
    - name: examine the password file
      command: cat /etc/passwd
      register: passwd_output

    - name: debug the output
      debug:
        msg: "{{ passwd_output.stdout_lines }}"
    when: debug_playbook

  - name: Loop over the list and create the accounts
    user:
      name: "{{ item }}"
      shell: "{{ my_shell }}"
    with_items: "{{ user_id_list }}"
    # when: item == "{{ my_user_id }}"
    become: true

  - name: a debug block of tasks
    block:
      - name: examine the password file
        command: cat /etc/passwd
        register: passwd_output

      - name: debug the output
        debug:
          msg: "{{ passwd_output.stdout_lines }}"
    when: debug_playbook

...
############################################################

ansible-playbook "ansible_conds_loops1.yml"

#### A more Complex Use of Loops
> Use a dictionary as the items in the loop
> - each part of the dictionary can be referenced separately
> - item.user_id - references the ID
> - item.full_name - references the Name of the user to add to the GECOS (Comments) field

   user_id_list:
    - { user_id: 'phcr', full_name: 'Philip Crawford' }
    - { user_id: 'mcpu', full_name: 'Matthew Chenette' }
    - { user_id: 'pdeg', full_name: 'Rose Oberlander' }
    - { user_id: 'nfgx', full_name: 'MJ Bautista' }

In [ ]:
wpb "ansible_conds_loops2.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    debug_playbook: True
    my_shell: "/bin/false"  # just using /bin/false as I dont want those accounts to be logged in
    my_user_id: "phcr"

    user_id_list:
        - { user_id: 'phcr', full_name: 'Philip Crawford' }
        - { user_id: 'mcpu', full_name: 'Matthew Chenette' }
        - { user_id: 'pdeg', full_name: 'Rose Oberlander' }
        - { user_id: 'nfgx', full_name: 'MJ Bautista' }

  tasks:

  - name: Loop over the list and create the accounts
    user:
      name: "{{ item.user_id }}"
      comment: "{{ item.full_name }}"
      shell: "{{ my_shell }}"
    with_items: "{{ user_id_list }}"
    become: true
    # when: item.user_id == "{{ my_user_id }}"

  - name: a debug block of tasks
    block:
    - name: examine the password file
      command: cat /etc/passwd
      register: passwd_output

    - name: debug the output
      debug:
        msg: "{{ passwd_output.stdout_lines }}"
    when: debug_playbook

...
############################################################

ansible-playbook "ansible_conds_loops2.yml"

### Real World Example 2:

Adding packages using yum in a loop

In [ ]:
wpb "ansible_loop_install.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    install_packages: True
    package_list:
      - tomcat
      - tomcat-webapps
      - tomcat-admin-webapps
      - tomcat-docs-webapp
      - tomcat-javadoc

  tasks:
    - name: Loop over list of packages to make sure they are not installed
      package:
        name: "{{ item }}"
        state: absent
      loop: "{{ package_list }}"
      become: true

    - name: show if tomcat installed # This should fail as tomcat is not installed
      command: "which tomcat"
      register: tomcat_check1
      ignore_errors: true

    - name: show the output of the check
      debug:
        msg: "{{ tomcat_check1.stdout }}"

    - name: Loop over the list of packages one by one to ensure they are installed
      package:
        name: "{{ item }}"
        state: latest
      with_items:
      - "{{ package_list }}"
      when:
      - install_packages
      become: true

    - name: show if tomcat installed
      command: "which tomcat"
      register: tomcat_check2

    - name: show the output of the check
      debug:
        msg: "{{ tomcat_check2.stdout_lines }}"

...
############################################################

time ansible-playbook "ansible_loop_install.yml"

#### But actually a recommended way is to use the entire list to install packages and not loop

> Note: I have added time to the commands to run the ansible playbook
> - It can be seen that this runs a few seconds faster
> - With longer lists of packages the time savings is considerable
> - Installing each item in turn in a loop the OS looks at each package indiviually and decides what dependencies are required
> - With a list the OS looks at the ebtire list and decides what dependencies are required

In [ ]:
wpb "ansible_loop_install2.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    install_packages: True
    package_list:
      - tomcat
      - tomcat-webapps
      - tomcat-admin-webapps
      - tomcat-docs-webapp
      - tomcat-javadoc

# But actually a recommended way is to use the entire list to install packages and not loop

  tasks:
    - name: Use the list to make sure the entire list is not installed
      package:
        name: "{{ package_list }}"
        state: absent
      become: true

    - name: show if tomcat installed # This should fail as tomcat is not installed
      command: "which tomcat"
      register: tomcat_check3
      ignore_errors: true

    - name: show the output of the check
      debug:
        msg: "{{ tomcat_check3.stdout }}"

    - name: Use the list to ensure they are installed
      package:
        name: "{{ package_list }}"
        state: latest
      when:
        - install_packages
      become: true

    - name: show if tomcat installed
      command: "which tomcat"
      register: tomcat_check4

    - name: show the output of the check
      debug:
        msg: "{{ tomcat_check4.stdout_lines }}"
...
############################################################

time ansible-playbook "ansible_loop_install2.yml"

## Further Explanation
---
**Examples using `block` and `include_tasks`:**
```yaml
# Using a conditional and a loop on an 'include_tasks'
- include_tasks: file_with_more_tasks.yml
  loop: "{{ example_ansible_list }}"
  when: boolean_variable

# Using a conditional on a block of tasks
- name: Block of tasks
  block:
    - name: Debug a Message
      debug:
        msg: "First task in the block."

    - name: Debug a Message
      debug:
        msg: "Second task in the block."
  when: boolean_variable
```

**See Also:**
- [Guide to Conditionals](https://docs.ansible.com/ansible/latest/user_guide/playbooks_conditionals.html)
- [Guide to Loops](https://docs.ansible.com/ansible/latest/user_guide/playbooks_loops.html)